# Aggregating Preferences: Social Choice

A simple example of the designer perspective is voting. How should a central authority pool the preferences of different agents so as to best reflect the wishes of the population as a whole? It turns out that voting, the kind familiar from our political and other institutions, is only a special case of the general class of _social choice problems_. Social choice is a motivational but nonstrategic theory—agents have preferences, but do not try to camouflage them in order to manipulate the outcome (of voting, for example) to ther personal advantage.  

An good example for social choice theory is plurality voting. Here we ask every voter to vote for his/her favorite choice after which the choice that received the largest number of votes is chosen. This is called the _plurality_ method. This method is quite standard but not without problems, because there is a need to select a tie-breaking rule (when two choices are equally favored).

### Condorcet condition
This condition states that if there exists a candidate $x$ such that for all other candidates $y$ at least half the voters prefer $x$ to $y$, then $x$ must be chosen.  

For example, with the following order in place:  
$$\begin{aligned} a &\succ b \succ c \\ b &\succ c \succ a \\ c &\succ b \succ a \end{aligned}$$
With the plurality method there would be a tie between all three candidates and so a tie breaker would be activated. However, the Condorcet condition would choose $b$, since two of the three candidates prefer $b$ to $a$ and likewise prefer $b$ to $c$.  
  
But the Condorcet isn't without problems. When the following order is in place:
$$\begin{aligned} a &\succ b \succ c \\ b &\succ c \succ a \\ c &\succ a \succ b \end{aligned}$$
The Condorcet condition does not tell what to do here, because there is no real preference displayed.

In [1]:
import operator as op
import random
import itertools
from typing import List
from dataclasses import dataclass

## Voting methods
Now some voting methods are displayed and explained

In [2]:
@dataclass
class Vote:
    choices: List[str]
    ranking: List[int]


class VoteCollection:
    def __init__(self):
        self.box = []
    
    def add_vote(self, vote: Vote):
        if len(vote.choices) == len(vote.ranking):
            self.box.append(vote)
        else:
            print("illegal vote")
    
    def make_count_register(self):
        register = {}

# Make the votes and fill up the votecollection
transformers = ["Optimus Prime", "Bumblebee", "Megatron", "Ironhide", "Ratchet"]
vote_amount = 200  # amount of total votes 


votes = VoteCollection()
for _ in range(vote_amount):
    votes.add_vote(Vote(random.sample(transformers, len(transformers)), [x for x in range(1, len(transformers)+1)]))

### plurality voting
_Each voter casts a single vote. The candidate with the most votes is selected._  
  
With this method ties must be broken according to a tie-breaking rule (e.g., a runoff election between the first-place candidates, alphabetic order of names, etc). 

In [3]:
def plurality_voting(votes: VoteCollection, verbose: bool = False):
    """Calculates the most favored choice based on the plurality method."""
    vote_register = {}
    for vote in votes.box:
        first_choice = vote.choices[0]
        if first_choice in vote_register:
            vote_register[first_choice] += 1
        else:
            vote_register[first_choice] = 1

    if verbose:
        for key, value in sorted(vote_register.items(), key=op.itemgetter(1), reverse=True):
            print(key, value)

    return f"Winner is: {max(vote_register.items(), key=(lambda k: vote_register[k[0]]))[0]}"

plurality_voting(votes, verbose=True)

Bumblebee 48
Ironhide 43
Megatron 42
Ratchet 40
Optimus Prime 27


'Winner is: Bumblebee'

### Cumulative voting
_Each voter is given k votes, which can be
cast arbitrarily (e.g., several votes could be cast for one candidate, with the remainder of the votes being distributed across other candidates). The candidate with the
most votes is selected._

### Approval voting
_Each voter can cast a single vote for as many of the candidates as he wishes; the candidate with the most votes is selected._  
  



### Plurality with elimination
_Each voter casts a single vote for their most-preferred candidate. The candidate with the fewest votes is eliminated. Each voter who cast a vote for the eliminated candidate casts a new vote for the candidate he most prefers among the candidates that have not been eliminated. This process is repeated until only one candidate remains._  
  


In [4]:
def plurwithelem_voting(votes: VoteCollection, verbose: int = False):
    """Calculates the favorite choice based on the plurality with elimination method."""
    available_candidates = list(set(itertools.chain.from_iterable([x.choices for x in votes.box])))

    while len(available_candidates) > 1:
        vote_register = {k:0 for k in available_candidates}

        # divide the votes
        for vote in votes.box:
            for pref in vote.choices:
                if pref in available_candidates:
                    vote_register[pref] += 1
                    break

        # count the votes and kick out the least favorite
        min_votes = min(vote_register.items(), key=(lambda k: vote_register[k[0]]))
        available_candidates.remove(min_votes[0])

        if verbose:
            print(f"Vote distribution:\n{vote_register}\nLoser is: {min_votes}\n")

    return f"Winner is: {available_candidates[0]}"

plurwithelem_voting(votes, verbose=True)

Vote distribution:
{'Optimus Prime': 27, 'Ratchet': 40, 'Megatron': 42, 'Bumblebee': 48, 'Ironhide': 43}
Loser is: ('Optimus Prime', 27)

Vote distribution:
{'Ratchet': 47, 'Megatron': 52, 'Bumblebee': 55, 'Ironhide': 46}
Loser is: ('Ironhide', 46)

Vote distribution:
{'Ratchet': 64, 'Megatron': 68, 'Bumblebee': 68}
Loser is: ('Ratchet', 64)

Vote distribution:
{'Megatron': 106, 'Bumblebee': 94}
Loser is: ('Bumblebee', 94)



'Winner is: Megatron'

### Borda voting
_Each voter submits a full ordering on the candidates. This ordering contributes points to each candidate; if there are $n$ candidates, it contributes $n − 1$ points to the highest ranked candidate, $n − 2$ points to the second highest, and so on; it contributes no points to the lowest ranked candidate. The winners are those whose total sum of points from all the voters is maximal._


In [5]:
def borda_voting(votes: VoteCollection, verbose: int = False):
    """Calculates the favorite choice based on the borda voting method."""
    available_candidates = list(set(itertools.chain.from_iterable([x.choices for x in votes.box])))
    n = len(available_candidates)
    vote_register = {k:0 for k in available_candidates}
    
    # contribute the points to each candidate
    for vote in votes.box:
        for candidate, rank in zip(vote.choices, vote.ranking):
            vote_register[candidate] += n - rank
        
    if verbose:
        print(vote_register)
    
    return f"Winner is: {max(vote_register.items(), key=(lambda k: vote_register[k[0]]))[0]}"

borda_voting(votes, verbose=True)

{'Optimus Prime': 372, 'Ratchet': 395, 'Megatron': 426, 'Bumblebee': 411, 'Ironhide': 396}


'Winner is: Megatron'

## Pairwise elimination
_In advance, voters are given a schedule for the order in which pairs of candidates will be compared. Given two candidates determine the candidate that each voter prefers. The candidate who is preferred by a minority of voters is eliminated, and the next pair of noneliminated candidates in the schedule is considered. Continue until only one candidate remains._


## Voting paradoxes

### Condorcet condition


### Sensitivity to a losing candidate


### Sensitivity to the agenda setter

